In [ ]:
import pandas as pd
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [103]:
import pandas as pd

# --- LOAD DATA ---
file_raw = 'DaftarSaham.xlsx'
file_summary = 'IDX Stock Summary 2020-2024.xlsx'

df_raw = pd.read_excel(file_raw, sheet_name=0)

# PENTING: Baca tanggal sebagai STRING dulu (jangan biarkan Excel auto-parse)
df_summary = pd.read_excel(file_summary, sheet_name=0, dtype={'Last Trading Date': str})

print("=== CEK FORMAT TANGGAL ASLI ===")
print(df_summary['Last Trading Date'].head(20))

# --- PERBAIKAN FORMAT TANGGAL ---
# Ganti nama bulan yang non-standard ke format standar
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace('agt', 'Aug', regex=False, case=False)
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace('Agt', 'Aug', regex=False)

# Convert ke datetime dengan format: "12 Dec 2022"
df_summary['Last Trading Date'] = pd.to_datetime(
    df_summary['Last Trading Date'], 
    format='%d %b %Y', 
    errors='coerce'
)

print(f"\n=== HASIL PARSING ===")
print(f"Sample tanggal setelah parsing:")
print(df_summary[['Stock Code', 'Last Trading Date']].dropna().head(10))
print(f"\nJumlah tanggal berhasil: {df_summary['Last Trading Date'].notna().sum()}")
print(f"Jumlah tanggal gagal (NaT): {df_summary['Last Trading Date'].isna().sum()}")

# --- TENTUKAN STATUS SAHAM ---
# Cari tanggal transaksi terakhir untuk tiap saham
last_transaction = df_summary.groupby('Stock Code')['Last Trading Date'].max()

# Referensi date: 16 Mei 2024 (hari ini/referensi saat analisis)
reference_date = pd.Timestamp('2024-05-16')

# Tentukan status: Listed jika transaksi terakhir pada/setelah 16 May 2024, Delisted jika sebelumnya
def get_status(last_date):
    if pd.isnull(last_date):
        return 'Delisted'
    return 'Listed' if last_date >= reference_date else 'Delisted'

status_mapping = last_transaction.apply(get_status)

# Debug: Cek berapa transaksi yang >= reference date
print(f"\n=== STATUS ANALISIS ===")
print(f"Jumlah saham dengan transaksi >= {reference_date.date()}: {(last_transaction >= reference_date).sum()}")
print(f"Jumlah saham dengan transaksi < {reference_date.date()}: {(last_transaction < reference_date).sum()}")

# --- MASUKKAN STATUS KE DATA UTAMA ---
df_raw['Status_Saham'] = df_raw['Code'].map(status_mapping)
df_raw['Status_Saham'] = df_raw['Status_Saham'].fillna('Delisted')

# --- TAMPILKAN SAMPLE RANDOM ---
print("\n--- SAMPLE RANDOM (10 data) ---")
print(df_raw[['Code', 'Name', 'Status_Saham']].sample(10))

print(f"\nTotal Listed: {(df_raw['Status_Saham'] == 'Listed').sum()}")
print(f"Total Delisted: {(df_raw['Status_Saham'] == 'Delisted').sum()}")

=== CEK FORMAT TANGGAL ASLI ===
0     02 Jan 2020
1     02 Jan 2020
2     02 Jan 2020
3     02 Jan 2020
4     02 Jan 2020
5     02 Jan 2020
6     02 Jan 2020
7     02 Jan 2020
8     02 Jan 2020
9     02 Jan 2020
10    02 Jan 2020
11    02 Jan 2020
12    02 Jan 2020
13    02 Jan 2020
14    02 Jan 2020
15    02 Jan 2020
16    02 Jan 2020
17    02 Jan 2020
18    02 Jan 2020
19    02 Jan 2020
Name: Last Trading Date, dtype: object

=== HASIL PARSING ===
Sample tanggal setelah parsing:
  Stock Code Last Trading Date
0       AALI        2020-01-02
1       ABBA        2020-01-02
2       ABDA        2020-01-02
3       ABMM        2020-01-02
4       ACES        2020-01-02
5       ACST        2020-01-02
6       ADES        2020-01-02
7       ADHI        2020-01-02
8       ADMF        2020-01-02
9       ADMG        2020-01-02

Jumlah tanggal berhasil: 831622
Jumlah tanggal gagal (NaT): 0

=== STATUS ANALISIS ===
Jumlah saham dengan transaksi >= 2024-05-16: 929
Jumlah saham dengan transaksi < 2024

In [93]:
import numpy as np

# --- 2. PEMBERSIHAN TANGGAL SUPER DETAIL ---
# Ubah ke string, huruf kecil semua, dan hapus spasi di depan/belakang
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].astype(str).str.lower().str.strip()

# Dictionary pengganti bulan (Indonesia -> Inggris)
# Kita pakai huruf kecil semua biar pasti kena
bulan_map = {
    'mei': 'may',
    'agt': 'aug',
    'ags': 'aug',  # Jaga-jaga kalau singkatan beda
    'okt': 'oct',
    'des': 'dec',
    'jan': 'jan', 'feb': 'feb', 'mar': 'mar', 'apr': 'apr',
    'jun': 'jun', 'jul': 'jul', 'sep': 'sep', 'nop': 'nov', 'nov': 'nov'
}

# Lakukan replace
for indo, eng in bulan_map.items():
    df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace(indo, eng, regex=False)

# Convert ke Datetime
# errors='coerce' akan mengubah yang error jadi NaT (Not a Time)
df_summary['Last Trading Date'] = pd.to_datetime(df_summary['Last Trading Date'], format='%d %b %Y', errors='coerce')

# --- CEK ERROR PARSING (PENTING!) ---
# Kita cek berapa baris yang tanggalnya jadi NaT (Gagal baca)
error_dates = df_summary[df_summary['Last Trading Date'].isna()]
print(f"Jumlah baris dengan tanggal error/kosong: {len(error_dates)}")
if len(error_dates) > 0:
    print("Contoh data yang error (Original):")
    # Tampilkan data asli kalau ada kolom original, atau tampilkan indexnya
    print(error_dates.head())

# --- 3. LOGIKA STATUS ---
# Cari tanggal transaksi TERAKHIR
last_trx = df_summary.groupby('Stock Code')['Last Trading Date'].max()

# Cutoff: 16 Mei 2024
cutoff_date = pd.Timestamp('2024-05-16')

# Fungsi Status
def get_status(last_date):
    if pd.isnull(last_date): 
        return 'Delisted' # Atau 'Check Data' jika ingin investigasi
    if last_date >= cutoff_date:
        return 'Listed'
    else:
        return 'Delisted'

status_mapping = last_trx.apply(get_status)

# --- 4. MASUKKAN KE DATA UTAMA ---
df_raw['Status_Saham'] = df_raw['Code'].map(status_mapping)
df_raw['Status_Saham'] = df_raw['Status_Saham'].fillna('Delisted')

# Cek Hasil dengan Sampel Acak
print("\n--- HASIL STATUS SAHAM SETELAH FIX ---")
print(df_raw[['Code', 'Status_Saham']].tail(50))

Jumlah baris dengan tanggal error/kosong: 831622
Contoh data yang error (Original):
   No Stock Code                  Company Name   Remarks  Previous  \
0   1       AALI       Astra Agro Lestari Tbk.  --S1K--1     14575   
1   2       ABBA             Mahaka Media Tbk.  --U9---2       106   
2   3       ABDA  Asuransi Bina Dana Arta Tbk.  --U8---2      6975   
3   4       ABMM            ABM Investama Tbk.  --U9---1      1530   
4   5       ACES   Ace Hardware Indonesia Tbk.  --S9KB-1      1495   

   Open Price Last Trading Date  First Trade   High    Low  ...    Bid  \
0           0               NaT            0  14650  14025  ...  14025   
1           0               NaT            0    108    105  ...    105   
2           0               NaT            0   7075   6925  ...   5600   
3           0               NaT            0      0      0  ...   1205   
4           0               NaT            0   1535   1500  ...   1510   

   Bid Volume  Listed Shares  Tradeble Shares  Wei

In [84]:
# --- 2. EKSPLORASI DATA AWAL ---
print("\n" + "="*50)
print("📊 PREVIEW DATA")
print("="*50)
print("\n🔍 Kolom di DaftarSaham:")
print(df_raw.columns.tolist())
print("\n🔍 Kolom di IDX Stock Summary:")
print(df_summary.columns.tolist())
print("\n🔍 Sample data Summary:")
print(df_summary.head(3))

# --- 3. PERBAIKAN FORMAT TANGGAL ---
print("\n" + "="*50)
print("🔧 MEMPERBAIKI FORMAT TANGGAL")
print("="*50)

# Cek tipe data kolom tanggal
print(f"Tipe data Last Trading Date sebelum: {df_summary['Last Trading Date'].dtype}")

# Convert ke string dulu
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].astype(str)

# Replace nama bulan Indonesia ke Inggris
month_mapping = {
    'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr',
    'Mei': 'May',  # Indonesia
    'Jun': 'Jun', 'Jul': 'Jul', 
    'Agt': 'Aug',  # Indonesia
    'Sep': 'Sep',
    'Okt': 'Oct',  # Indonesia
    'Nov': 'Nov',
    'Des': 'Dec'   # Indonesia
}

for indo, eng in month_mapping.items():
    df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace(
        indo, eng, regex=False
    )

# Convert ke datetime
df_summary['Last Trading Date'] = pd.to_datetime(
    df_summary['Last Trading Date'], 
    format='%d %b %Y', 
    errors='coerce'
)

print(f"Tipe data Last Trading Date sesudah: {df_summary['Last Trading Date'].dtype}")
print(f"Jumlah tanggal yang gagal diparse (NaT): {df_summary['Last Trading Date'].isna().sum()}")

# --- 4. LOGIKA STATUS SAHAM ---
print("\n" + "="*50)
print("🎯 MENENTUKAN STATUS SAHAM")
print("="*50)

# Cari tanggal transaksi TERAKHIR untuk setiap Stock Code
last_transaction = df_summary.groupby('Stock Code')['Last Trading Date'].max()

print(f"\n📌 Total unique stock codes di data summary: {len(last_transaction)}")
print(f"📌 Total stock codes di DaftarSaham: {len(df_raw)}")

# Tentukan cutoff date
cutoff_date = pd.Timestamp('2024-05-16')
print(f"\n📅 Cutoff Date: {cutoff_date.strftime('%d %B %Y')}")

# Fungsi untuk menentukan status
def determine_status(last_date):
    """
    Logika:
    - Jika tanggal terakhir >= 16 Mei 2024 → Listed
    - Jika tanggal terakhir < 16 Mei 2024 → Delisted
    - Jika tanggal tidak ada (NaT/None) → Delisted
    """
    if pd.isnull(last_date):
        return 'Delisted'
    return 'Listed' if last_date >= cutoff_date else 'Delisted'

# Apply fungsi ke setiap saham
status_mapping = last_transaction.apply(determine_status)

# --- 5. MASUKKAN STATUS KE DATA UTAMA ---
print("\n" + "="*50)
print("📝 MENGGABUNGKAN STATUS KE DAFTARSAHAM")
print("="*50)

# Map status berdasarkan Stock Code
df_raw['Status_Saham'] = df_raw['Code'].map(status_mapping)

# Isi yang kosong dengan 'Delisted' (untuk saham yang tidak ada di data summary)
df_raw['Status_Saham'] = df_raw['Status_Saham'].fillna('Delisted')

# --- 6. HASIL DAN STATISTIK ---
print("\n" + "="*50)
print("📈 HASIL ANALISIS STATUS SAHAM")
print("="*50)

# Hitung jumlah Listed vs Delisted
status_count = df_raw['Status_Saham'].value_counts()
print(f"\n✅ Listed: {status_count.get('Listed', 0)} saham")
print(f"❌ Delisted: {status_count.get('Delisted', 0)} saham")
print(f"📊 Total: {len(df_raw)} saham")

# Persentase
if len(df_raw) > 0:
    pct_listed = (status_count.get('Listed', 0) / len(df_raw)) * 100
    pct_delisted = (status_count.get('Delisted', 0) / len(df_raw)) * 100
    print(f"\n📊 Persentase Listed: {pct_listed:.2f}%")
    print(f"📊 Persentase Delisted: {pct_delisted:.2f}%")

# Sample data hasil
print("\n" + "="*50)
print("🔍 SAMPLE HASIL (10 Data Acak)")
print("="*50)
print(df_raw[['Code', 'Name', 'Status_Saham']].sample(min(10, len(df_raw))))

# Sample Listed dan Delisted
print("\n📌 Sample saham LISTED:")
listed_sample = df_raw[df_raw['Status_Saham'] == 'Listed'][['Code', 'Name']].head(5)
print(listed_sample)

print("\n📌 Sample saham DELISTED:")
delisted_sample = df_raw[df_raw['Status_Saham'] == 'Delisted'][['Code', 'Name']].head(5)
print(delisted_sample)


📊 PREVIEW DATA

🔍 Kolom di DaftarSaham:
['Code', 'Name', 'ListingDate', 'Shares', 'ListingBoard', 'Sector', 'LastPrice', 'MarketCap', 'Status_Saham', 'PreviousPrice 2020', 'LastPrice 2024', 'Highest Volume Trading', 'Date Highest Volume Trading', 'Close 2020', 'Close 2021', 'Close 2022', 'Close 2023', 'Close 2024', 'CAGR 2020 to 2024']

🔍 Kolom di IDX Stock Summary:
['No', 'Stock Code', 'Company Name', 'Remarks', 'Previous', 'Open Price', 'Last Trading Date', 'First Trade', 'High', 'Low', 'Close', 'Change', 'Volume', 'Value', 'Frequency', 'Index Individual', 'Offer', 'Offer Volume', 'Bid', 'Bid Volume', 'Listed Shares', 'Tradeble Shares', 'Weight For Index', 'Foreign Sell', 'Foreign Buy', 'Non Regular Volume', 'Non Regular Value', 'Non Regular Frequency']

🔍 Sample data Summary:
   No Stock Code                  Company Name   Remarks  Previous  \
0   1       AALI       Astra Agro Lestari Tbk.  --S1K--1     14575   
1   2       ABBA             Mahaka Media Tbk.  --U9---2       106   

In [10]:
# --- 2. PERBAIKAN TANGGAL (FIX INDONESIAN MONTHS) ---
# Mengubah kolom menjadi string dulu biar bisa di-replace
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].astype(str)

# Ganti nama bulan Indonesia ke Inggris
# Tips: Pakai regex=False biar aman
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace('Mei', 'May', regex=False)
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace('Agt', 'Aug', regex=False)
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace('Okt', 'Oct', regex=False)
df_summary['Last Trading Date'] = df_summary['Last Trading Date'].str.replace('Des', 'Dec', regex=False)

# Convert ke Datetime
# errors='coerce' artinya kalau ada data tanggal yang aneh/kosong, dia bakal jadi NaT (Not a Time) dan gak bikin error
df_summary['Last Trading Date'] = pd.to_datetime(df_summary['Last Trading Date'], format='%d %b %Y', errors='coerce')

In [81]:
# --- 3. LOGIKA STATUS SAHAM (LISTED / DELISTED) ---
# Cari tanggal transaksi TERAKHIR untuk setiap saham
last_trx = df_summary.groupby('Stock Code')['Last Trading Date'].max()

# Tentukan Cutoff: 16 Mei 2024
cutoff_date = pd.Timestamp('2024-05-16')

# Bikin mapping status
def get_status(last_date):
    if pd.isnull(last_date): # Kalau tanggalnya kosong/gak ketemu
        return 'Delisted'
    if last_date >= cutoff_date:
        return 'Listed'
    else:
        return 'Delisted'

status_mapping = last_trx.apply(get_status)

In [82]:
# --- 4. MASUKKAN KE DATA UTAMA ---
df_raw['Status_Saham'] = df_raw['Code'].map(status_mapping)

# Isi yang kosong (jika ada saham di df_raw yang gak ada di data transaksi)
df_raw['Status_Saham'] = df_raw['Status_Saham'].fillna('Delisted')

# Cek Hasil
print("\n--- HASIL STATUS SAHAM ---")
print(df_raw[['Code', 'Status_Saham']].head(1000))

# Menampilkan 5 sampel acak dari data
print("\n--- SAMPLE ACAK DATA ---")
print(df_raw[['Code', 'Name', 'Status_Saham']].sample(5))


--- HASIL STATUS SAHAM ---
     Code Status_Saham
0    AALI       Listed
1    ABBA       Listed
2    ABDA       Listed
3    ABMM       Listed
4    ACES       Listed
..    ...          ...
824  ZATA       Listed
825  ZBRA       Listed
826  ZINC       Listed
827  ZONE       Listed
828  ZYRX       Listed

[829 rows x 2 columns]

--- SAMPLE ACAK DATA ---
     Code                           Name Status_Saham
84   BBKP           Bank KB Bukopin Tbk.       Listed
110  BIKA      Binakarya Jaya Abadi Tbk.       Listed
582  PGJO  Tourindo Guide Indonesia Tbk.       Listed
782  TURI              Tunas Ridean Tbk.     Delisted
737  TBLA        Tunas Baru Lampung Tbk.       Listed
